In [1]:
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from storage_utils.patients_data_new import patients
import os

/home/franciscosargo/.virtualenvs/pre_epi_seizures/local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# Load Baseline Removal
disk='/mnt/Seagate/'
files_dir = 'h5_files_backup/processing_datasets/'
file_name = 'seizure_datasets_new.h5'

file_path = disk + files_dir + file_name

print file_path

# patient parameters
patient_nr=3
patient_info = patients[str(patient_nr)]
patient_info

# feature for analysis
feature_name=('signals/3000_1200'
              '/raw'
              '/baseline_removal__filt:MedianFIR')

# load the file
f = h5py.File(file_path, 'r')

print feature_name

seizures = f[feature_name].keys()

# get seizures from patient
seizure_from_patient = [seizure
                         for seizure in seizures
                         if seizure.split('_')[0] == str(patient_nr)]
print seizure_from_patient

patient_info

# Create a list of types of seizure to join
types_of_seizure = patient_info['types_of_seizure'] + patient_info['types_of_seizure']


print types_of_seizure

# Set path to save
path_to_save = '/home/franciscosargo/final_results/PATIENT{}/baseline_removal/'.format(patient_nr) 
if not os.path.exists(path_to_save):
    print 'File doesnt exist'
    os.makedirs(path_to_save)
    
    
# Get the time parameters of the signal
Fs=1000
sec_before_onset = 50 * 60
sec_after_onset = 20 * 60
time_seconds = np.linspace(-sec_before_onset, sec_after_onset, 
                           (sec_before_onset + sec_after_onset) * Fs)
    
# get the hdf5 dataset
seizure_hdfdataset = f[feature_name + '/' + seizure_name]

# load the array to memory
seizure_array = seizure_hdfdataset[:]


    

In [ ]:
# Compute HRV signal
hrv_seizure = 

In [ ]:
seizure_features=['Removed Baseline and filtered']


# Plot the signal
plt.figure(figsize=(20, 20))
plt.plot(time_seconds, seizure_array.T)
plt.axvline(x=0, c='k')
plt.legend([seizure_features[0], 'EEG Onset'])
plt.xlabel('Relative Time to onset [s]')
plt.ylabel('Digitation Units')
plt.title(seizure_name + ' ' + '|' + ' ' + type_of_seizure)
plt.show() 

# Get seizure id
seizure_id = seizure_name.split('_')
lead_name = seizure_id[2]
seizure_nr = seizure_id[-1]
patient_nr = seizure_id[0]



plt.savefig(path_to_save + patient_nr + '_' + seizure_nr + '_' + lead_name + '.png')